In [ ]:
import helpers.ansatz as torch_ansatz
import helpers.ansatz_JAX as jax_ansatz

import torch
import numpy as np
import jax.numpy as jnp
import matplotlib.pyplot as plt
import math

In [ ]:


rng = np.random.default_rng(seed=123)

mstar   = 0
max_M   = 4          # rows   (≥ 3 if you want α² terms)
max_N   = 5          # columns
B       = 6          # batch size  for t

alpha   = 0.37

t       = torch.tensor(rng.uniform(0.2, 4.0, size=(B,)), dtype=torch.float32)
t_jax   = jnp.asarray(np.array(t))

g_coeffs_np = rng.normal(scale=0.1, size=(max_M, max_N)).astype(np.float32)
theta_np    = rng.normal(scale=0.3, size=(max_M, 1   )).astype(np.float32)

g_coeffs_torch = torch.tensor(g_coeffs_np)
theta_torch    = torch.tensor(theta_np)
g_coeffs_jax   = jnp.asarray(g_coeffs_np)
theta_jax      = jnp.asarray(theta_np)

# ----- parameters for q -------------------------------------------------
t_min = 0.0
t_max = 6.0

factorial_cache_n_np = np.array([math.factorial(k) for k in range(max_N)],
                                dtype=np.float32)
factorial_cache_m_np = np.array([math.factorial(k)
                                 for k in range(mstar, max_M + mstar)],
                                dtype=np.float32)
n_range_np = np.arange(max_N, dtype=np.int32)
m_range_np = np.arange(1, max_M, dtype=np.int32)

factorial_cache_info_torch = (
    torch.tensor(factorial_cache_m_np),
    torch.tensor(factorial_cache_n_np),
    torch.tensor(m_range_np),
    torch.tensor(n_range_np),
)

factorial_cache_info_jax = (
    jnp.asarray(factorial_cache_m_np),
    jnp.asarray(factorial_cache_n_np),
    jnp.asarray(m_range_np),
    jnp.asarray(n_range_np),
)

